# Demo

## 1. Set uṕ a data source for your knowledge base 

Here is a table with the requested information:

| Format                        | Extension    |
|-------------------------------|--------------|
| Plain text                    | .txt         |
| Markdown                      | .md          |
| HyperText Markup Language     | .html        |
| Microsoft Word document       | .doc/.docx   |
| Comma-separated values        | .csv         |
| Microsoft Excel spreadsheet   | .xls/.xlsx   |
| Portable Document             | .pdf         |


<br/>

> The data is splited into **chunks**, which is an extract from a data source that is returned when the KB that it belongs to is queried 

### 3 types of chunks: 

1. **Default chunking**: By default, Amazon Bedrock automatically splits your source data into chunks, such that each chunk contains, at most, approximately 300 tokens. If a document contains less than 300 token, then it is not split any further
2. **Fixed size chunking**: Amazon Bedrock splits your source data into chunks of the approximate size that you set.
3. **No chunking**: Amazon Bedrock treats each file as one chunk. If you choose this option, you may want to pre-process your documents by splitting them into separate files before uploading them to an Amazon S3 bucket.

In [ ]:
# 0. Create a S3 Bucket and upload content

#TODO: Do it from code (this time it was uploaded directly to focus on Bedrock)

In [29]:
import boto3
import json
region_name = 'us-east-1'  # Update this with your desired region

brt = boto3.Session(profile_name="sebas", region_name=region_name).client(service_name='bedrock-runtime')

modelId = "anthropic.claude-v2:1"
contentType = 'application/json'
accept = 'application/json'
human_prompt = """
Give an outline of 10 chapters for a ebook of AWS services for beginners. 
The output should be a JSON with the chapter titles and a brief description of each chapter.
the JSON structure should be the following:
{
    "chapters": [
        {
            "number": int, 
            "title": string, 
            "description": string
        },

    ]
}
"""
body = json.dumps({
    'prompt': f'\n\nHuman:{human_prompt}\n\nAssistant:',
    "max_tokens_to_sample": 2000,
    "temperature": 0.5,
    "top_p": 1,
})

response = brt.invoke_model(
    body=body, 
    modelId=modelId, 
    accept=accept, 
    contentType=contentType
)

response_body = json.loads(response.get('body').read())

In [30]:
import json

# Specify the file path
file_path = "file.json"

# Write the response_body to the JSON file
with open(file_path, "w") as file:
    json.dump(response_body['completion'], file)


In [31]:
reply_from_claude = response_body['completion']
print(reply_from_claude)

 Here is an outline of 10 chapters for an ebook of AWS services for beginners:

```json
{
    "chapters": [
        {
            "number": 1,
            "title": "Introduction to AWS",
            "description": "A brief overview of AWS and its key services and concepts."
        },
        {
            "number": 2,
            "title": "AWS Global Infrastructure", 
            "description": "Understanding AWS regions, availability zones and edge locations."
        },
        {
            "number": 3,
            "title": "AWS EC2",
            "description": "Provisioning and managing virtual servers using Amazon Elastic Compute Cloud."
        },
        {
            "number": 4,
            "title": "AWS S3",
            "description": "Using Amazon Simple Storage Service for cloud object storage." 
        },
        {
            "number": 5,
            "title": "AWS Database Services",
            "description": "Managed database options like RDS, DynamoDB and Redshift."


In [32]:
import json

# Extract the JSON content
json_start = reply_from_claude.find("```json") + len("```json")
json_end = reply_from_claude.find("```", json_start)
json_content = reply_from_claude[json_start:json_end]

# Remove newlines from the JSON content
json_content = json_content.replace("\n", "")

# Parse the JSON content into a dictionary
data = json.loads(json_content)

# Print the dictionary
print(data)



{'chapters': [{'number': 1, 'title': 'Introduction to AWS', 'description': 'A brief overview of AWS and its key services and concepts.'}, {'number': 2, 'title': 'AWS Global Infrastructure', 'description': 'Understanding AWS regions, availability zones and edge locations.'}, {'number': 3, 'title': 'AWS EC2', 'description': 'Provisioning and managing virtual servers using Amazon Elastic Compute Cloud.'}, {'number': 4, 'title': 'AWS S3', 'description': 'Using Amazon Simple Storage Service for cloud object storage.'}, {'number': 5, 'title': 'AWS Database Services', 'description': 'Managed database options like RDS, DynamoDB and Redshift.'}, {'number': 6, 'title': 'AWS Networking and Content Delivery', 'description': 'Services like VPC, CloudFront, Route 53 and API Gateway.'}, {'number': 7, 'title': 'AWS Security, Identity & Compliance', 'description': 'IAM roles, security groups, AWS Organizations and compliance standards.'}, {'number': 8, 'title': 'AWS Management Tools', 'description': 'M

In [33]:
file_path = "outline.json"
with open(file_path, "w") as file:
    json.dump(data, file)


In [1]:
import requests
import json

url = 'https://vl3cnc62bg.execute-api.us-east-1.amazonaws.com/dev'

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Ck07u8rQEa8fyz8h1NmwA7ebVvb3Gdtw9cOijwdF'

}
prompt = "Hello, I am a human."

data = {
    "prompt": prompt,

}

# Make a POST request with custom headers
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response status code and content
print('Response Status Code:', response.status_code)
print('Response Content:', response.text)


Response Status Code: 403
Response Content: {"message":"Authorization header requires 'Credential' parameter. Authorization header requires 'Signature' parameter. Authorization header requires 'SignedHeaders' parameter. Authorization header requires existence of either a 'X-Amz-Date' or a 'Date' header. Authorization=Ck07u8rQEa8fyz8h1NmwA7ebVvb3Gdtw9cOijwdF"}


In [13]:
import boto3

# Create an AWS SSO session
session = boto3.Session(profile_name='sebas')

# Use the session to make API requests
apigateway_client = session.client('apigateway')


In [19]:
rest_api_id = 'vl3cnc62bg'
resource_id = 'qdd990'
http_method = 'POST'
path_with_query_string = 'prompt = "Hello"'
#payload = {'prompt': 'Hello'}

response = apigateway_client.test_invoke_method(
    restApiId=rest_api_id,
    resourceId=resource_id,
    httpMethod=http_method,
    pathWithQueryString=path_with_query_string,
    #body=json.dumps(payload)
)

# Process the API response
print('Response Status Code:', response['status'])
print('Response Body:', response['body'])

Response Status Code: 200
Response Body: {"errorMessage": "An error occurred (ValidationException) when calling the InvokeModel operation: Input is too long for requested model.", "errorType": "ValidationException", "requestId": "71cb6b33-9e2c-4386-be8e-6a67524a2091", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 14, in lambda_handler\n    client_bedrockrequest=client_bedrock.invoke_model(\n", "  File \"/var/lang/lib/python3.12/site-packages/botocore/client.py\", line 535, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/lang/lib/python3.12/site-packages/botocore/client.py\", line 980, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}
